# RFM Analysis (Python)

This notebook performs RFM (Recency, Frequency, Monetary) segmentation on a retail dataset.
We will load transactional data, compute RFM metrics, assign RFM scores, and prepare a dataset for Power BI.

## 1. Load Libraries

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime as dt

## 2. Load Dataset

In [ ]:
# Update path based on your repo structure
DATA_PATH = Path("../data/Sample - Superstore.csv")
df = pd.read_csv(DATA_PATH)
df.head()

## 3. Prepare Data for RFM

In [ ]:
# Convert Order Date to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Reference date (max date in dataset)
reference_date = df['Order Date'].max() + pd.Timedelta(days=1)
reference_date

## 4. Calculate RFM Metrics

In [ ]:
# RFM calculation
rfm = df.groupby('Customer Name').agg({
    'Order Date': lambda x: (reference_date - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm.head()

## 5. Score RFM (1–4)

In [ ]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1,2,3,4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1,2,3,4])

rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm.head()

## 6. Assign Segments

In [ ]:
def segment_customer(row):
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])

    if r >= 3 and f >= 3 and m >= 3:
        return 'Champion'
    if r >= 3 and f >= 2:
        return 'Loyal'
    if r == 4:
        return 'Recent'
    if f == 4:
        return 'Frequent'
    if m == 4:
        return 'High Spender'
    return 'Others'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)
rfm.head()

## 7. Export RFM Table for Power BI

In [ ]:
OUTPUT_PATH = Path("../data/RFM_Table.csv")
rfm.to_csv(OUTPUT_PATH)
OUTPUT_PATH